## Trabajo 02 Renta fija Sql y python - Laura Rojas - Edwin Rios

#### Librerias

In [1]:
import mysql.connector
import datetime as dt
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt


In [2]:
import os
import sys
import inspect

In [3]:
import Clase_6.Funciones as funciones
from Clase_6.Funciones import DB_class
import Clase_6.Clase_6Fun as clase_6Fun

--funciones\
    |-DB_class\
    |-coupon_m(parametros)\
    |-discount_factor_m(parametros)\
    |-dirty_price_m(parametros)\
    |-dirty_price_1i(parametros,inst_pos)\
    |-ytm_1d_obj_func(x,i,params,prices)\
    |-find_ytm_loop(parametros,prices)\
    |-ytm_obj_func(x,params,prices)\
    |-find_ytm_m(parametros,prices)\
    |-NelsonSiegel_1d(params, ven)\
    |-NelsonSiegel_m(params, ven)\
    |-dirty_price_m_ns(betas,parametros)\
    |-objetive_function(betas,parametros)\
    |-betas_optimization(parametros)\
    |-betas_optimization_bound(parametros)\

--Clase_6Fun\
    |-Macaulay_duration_mat(parametros,prices)\
    |-Modified_duration_mat(parametros,prices)\
    |-Modified_duration_mat_2(parametros,prices,macaulay_duration)\
    |-Convexity_mat(parametros,prices)\
    |-Macaulay_duration_mat_loop(parametros,prices)\
    |-Modified_duration_mat_loop(parametros,prices)\
    |-Modified_duration_mat_loop_2(parametros,prices,macaulay_duration)\
    |-Convexity_mat_loop(parametros,prices)\
    |-objetive_function_prices(betas,parametros)\
    |-betas_optimization_bound_prices(parametros)\

In [4]:
import itertools
import importlib

#### Inclusión desde base de datos

In [15]:
db_object = DB_class(host="localhost",user="root",password="root",database="trabajo_1")
class2_db = db_object.open_db()
mi_cursor = class2_db.cursor()
query = ("SELECT * FROM trabajo_2.tabla_precios WHERE trabajo_2.tabla_precios.frecuencia_pago=1 "+\
    "order by trabajo_2.tabla_precios.Tipo, trabajo_2.tabla_precios.Indice_tipo")
tes_faciales = pd.read_sql(query, con=class2_db)
display(tes_faciales)

C:\Users\Edwin\AppData\Roaming\Python\Python39\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Tipo,Indice_tipo,fecha_valoracion,nemotecnico,fecha_emision,fecha_vencimiento,cupon,tir,frecuencia_pago,Indicador_indexado,conteo_dias,precio_sucio,duracion_mac,duracion_mod,Convexidad
0,T4,1,2022-09-18,CDTTGSS1V,2021-12-27,2022-09-27,7.1,4.538834,1,No,REAL/365,0.0,0.0,0.0,0.0
1,T4,2,2022-09-18,CDTAQCS1V,2021-09-28,2022-09-28,8.1,4.540110,1,No,REAL/365,0.0,0.0,0.0,0.0
2,T4,3,2022-09-18,CDTJXZS1V,2022-06-28,2022-09-28,6.1,4.540110,1,No,REAL/365,0.0,0.0,0.0,0.0
3,T4,4,2022-09-18,CDTFYMS1V,2022-03-29,2022-09-29,6.1,4.541385,1,No,REAL/365,0.0,0.0,0.0,0.0
4,T4,5,2022-09-18,CDTEGLS1V,2021-09-29,2022-09-29,6.1,4.541385,1,No,REAL/365,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,T4,96,2022-09-18,CDTAZMS1V,2022-06-07,2025-06-07,6.1,5.606767,1,No,REAL/365,0.0,0.0,0.0,0.0
96,T4,97,2022-09-18,CDTZVBS1V,2022-06-15,2025-06-15,7.1,5.614064,1,No,REAL/365,0.0,0.0,0.0,0.0
97,T4,98,2022-09-18,CDTBVTS1V,2022-09-09,2025-09-09,6.1,5.691255,1,No,REAL/365,0.0,0.0,0.0,0.0
98,T4,99,2022-09-18,CDTFVBS1V,2022-09-09,2025-09-09,7.1,5.691255,1,No,REAL/365,0.0,0.0,0.0,0.0


In [25]:
db_object = DB_class(host="localhost",user="root",password="root",database="trabajo_1")
class2_db = db_object.open_db()
mi_cursor = class2_db.cursor()
query = (
'SELECT '+\
'trabajo_2.tabla_precios.Tipo,'+\
'trabajo_2.tabla_precios.Indice_tipo,'+\
'trabajo_2.tabla_precios.fecha_valoracion,'+\
'trabajo_2.tabla_precios.nemotecnico,'+\
'trabajo_2.tabla_precios.fecha_emision,'+\
'trabajo_2.tabla_precios.fecha_vencimiento,'+\
'trabajo_2.tabla_precios.cupon,'+\
'trabajo_2.tabla_precios.tir,'+\
'trabajo_2.tabla_precios.frecuencia_pago,'+\
'trabajo_2.tabla_precios.Indicador_indexado,'+\
'trabajo_2.tabla_precios.conteo_dias,'+\
'trabajo_2.tabla_indices.Indice_c,'+\
'trabajo_2.tabla_indices.fecha_indice,'+\
'trabajo_2.tabla_indices.valor '+\
'FROM trabajo_2.tabla_precios '+\
'left join trabajo_2.tabla_indices '+\
'on '+\
'concat(trabajo_2.tabla_indices.Indice_c,trabajo_2.tabla_indices.fecha_indice) ='+\
'concat(trabajo_2.tabla_precios.Indicador_indexado,trabajo_2.tabla_precios.fecha_valoracion) '+\
'where '+\
'trabajo_2.tabla_precios.frecuencia_pago=12 and trabajo_2.tabla_indices.Indice_c=\'IPC\' '+\
'order by trabajo_2.tabla_precios.Tipo, trabajo_2.tabla_precios.Indice_tipo'
)
tes_faciales = pd.read_sql(query, con=class2_db)
display(tes_faciales)

C:\Users\Edwin\AppData\Roaming\Python\Python39\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Tipo,Indice_tipo,fecha_valoracion,nemotecnico,fecha_emision,fecha_vencimiento,cupon,tir,frecuencia_pago,Indicador_indexado,conteo_dias,Indice_c,fecha_indice,valor
0,T2,1,2022-09-18,CDTWDF90,2012-09-11,2022-10-12,4.02,2.160716,12,IPC,REAL/365,IPC,2022-09-18,10.84
1,T2,2,2022-09-18,CDTRZF90,2020-09-11,2022-12-12,2.15,2.533970,12,IPC,REAL/365,IPC,2022-09-18,10.84
2,T2,3,2022-09-18,CDTRDW90,2018-01-18,2023-01-18,2.90,2.747923,12,IPC,REAL/365,IPC,2022-09-18,10.84
3,T2,4,2022-09-18,CDTCNP90,2021-01-22,2023-01-22,0.65,2.770512,12,IPC,REAL/365,IPC,2022-09-18,10.84
4,T2,5,2022-09-18,CDTKVM90,2021-07-22,2023-01-22,0.50,2.770512,12,IPC,REAL/365,IPC,2022-09-18,10.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,T2,96,2022-09-18,CDTGCN90,2018-09-26,2043-09-26,4.12,7.884695,12,IPC,REAL/365,IPC,2022-09-18,10.84
96,T2,97,2022-09-18,CDTPAA90,2018-10-19,2043-10-19,4.11,7.885051,12,IPC,REAL/365,IPC,2022-09-18,10.84
97,T2,98,2022-09-18,CDTFIK90,2018-11-16,2043-11-16,4.11,7.885482,12,IPC,REAL/365,IPC,2022-09-18,10.84
98,T2,99,2022-09-18,CDTVQT90,2018-03-07,2044-03-07,4.10,7.887171,12,IPC,REAL/365,IPC,2022-09-18,10.84


In [37]:
val_date= tes_faciales.fecha_valoracion.drop_duplicates()[0]
ini_date = tes_faciales.fecha_emision
fin_date = tes_faciales.fecha_vencimiento
cpn =  tes_faciales.cupon
discount_rates = tes_faciales.tir
frec_pag=tes_faciales.frecuencia_pago
indicador=tes_faciales.valor
discount_rates_ind=((1+discount_rates/100)*(1+indicador/100)-1)*100

In [36]:
type(resultado)

pandas.core.series.Series

In [35]:
resultado=((1+discount_rates/100)*(1+indicador/100)-1)*100

In [39]:
### titulos
mul_inst = 1 #intentar 1, 100 y 10000
parametros = dict({
        'val_date':val_date, 
        'ini_date':np.tile(ini_date,mul_inst), 
        'fin_date':np.tile(fin_date,mul_inst), 
        'coupon':np.tile(cpn,mul_inst),
        'discount_rates':np.tile(discount_rates,mul_inst),
        'discount_rates_ind':np.tile(discount_rates_ind,mul_inst),
        'frec_pag':np.tile(frec_pag,mul_inst),
        'indicador':np.tile(indicador,mul_inst),
        'limits':np.array([-20,20])
        })

In [90]:
ini_date.shape

(200,)

In [40]:
importlib.reload(funciones)

<module 'Clase_6.Funciones' from 'd:\\cursos\\Maestria finanzas\\S3_PySql rent fij\\Trabajo02\\Clase_6\\Funciones.py'>

In [41]:
n_inst = len(parametros['coupon'])
## Funcion de matriz de cupones
coupon_dict = funciones.coupon_m365ind(parametros)
parametros['cpn_dict'] = coupon_dict
parametros['delta_t'] = coupon_dict['days_matrix']/365
parametros['discount_factors'] = funciones.discount_factor_m(parametros)

prices = funciones.dirty_price_m(parametros)

In [27]:
x_0=0
for i in range(10):
    x_0=+i*12
    print(x_0)

0
12
24
36
48
60
72
84
96
108


In [120]:
tes_faciales.fecha_emision[0]
issue=dt.datetime.strptime(str(tes_faciales.fecha_emision[0]),'%Y-%m-%d').date()
periodos=10
frecuencia_0=3
years_0=0
months_0=0
arr = np.array([])
for i in range(periodos):
    if i==0:
        years_0=0
    elif (frecuencia_0*i)%12==0:
        years_0=(frecuencia_0*i)/12
    months_0=(frecuencia_0*i)%12
    
    respuesta=issue + relativedelta(years=years_0) +relativedelta(months=months_0) 
    arr = np.append(arr, respuesta)    
    #print(years_0,' - ',months_0,' - ',respuesta)
print(arr)

[datetime.date(2012, 9, 11) datetime.date(2012, 12, 11)
 datetime.date(2013, 3, 11) datetime.date(2013, 6, 11)
 datetime.date(2013, 9, 11) datetime.date(2013, 12, 11)
 datetime.date(2014, 3, 11) datetime.date(2014, 6, 11)
 datetime.date(2014, 9, 11) datetime.date(2014, 12, 11)]


In [121]:
arr.shape

(10,)

In [87]:
tes_faciales.fecha_emision[0]
issue=dt.datetime.strptime(str(tes_faciales.fecha_emision[0]),'%Y-%m-%d').date()
periodos=10
for i in range(periodos):
    print(issue + relativedelta(years=+i))

2012-04-12
2013-04-12
2014-04-12
2015-04-12
2016-04-12
2017-04-12
2018-04-12
2019-04-12
2020-04-12
2021-04-12


In [78]:
tes_faciales.fecha_emision[0]

datetime.date(2012, 4, 12)

In [79]:
issue=dt.datetime.strptime(str(tes_faciales.fecha_emision[0]),'%Y-%m-%d').date()

In [80]:
#respuesta=issue + relativedelta(months=+1) 
respuesta=issue + relativedelta(years=0) +relativedelta(months=6) 
print(respuesta)

2012-10-12


In [ ]:
coupon_dates = np.array([issue + relativedelta(years=1,month=0) for x in range(int(flow))])

In [71]:
from datetime import datetime

In [76]:
now = datetime.now()
now = dt.datetime.strptime(str(datetime.now()),'%Y-%m-%d').date()
print(now)  # 👉️ 2022-06-22 19:27:37.483564

result = now + relativedelta(months=+3)

print(result)  # 👉️ 2022-09-22 19:27:37.483564

ValueError: unconverted data remains:  23:18:22.913338

In [112]:
prueba=np.array([1,2,2])

In [113]:
prueba.shape

(3,)

In [116]:
 
# creating an array
arr = np.array([1, 8, 3, 3, 5])
print('Original Array : ', arr)
 
# appending to the array
arr = np.append(arr, [7])
print('Array after appending : ', arr)

Original Array :  [1 8 3 3 5]
Array after appending :  [1 8 3 3 5 7]


In [117]:
arr.shape

(6,)

In [118]:
# creating an array
arr = np.array([])
print('Original Array : ', arr)
 
# appending to the array
arr = np.append(arr, [7])
print('Array after appending : ', arr)

Original Array :  []
Array after appending :  [7.]
